# **Chapter 49: Dart Language Reference**

---

## **Learning Objectives**

By the end of this chapter, you will have a comprehensive quick reference for:
- Dart syntax patterns and idioms used in Flutter development
- Null safety migration strategies and best practices
- Effective Dart style guidelines for maintainable code
- Asynchronous programming patterns and Stream manipulation

---

## **49.1 Quick Syntax Cheat Sheet**

### **Variables and Types**

```dart
// Type inference (preferred)
var name = 'Flutter';           // String
final age = 25;                 // int (runtime constant)
const pi = 3.14159;             // Compile-time constant

// Explicit types
String title = 'App';
int count = 0;
double price = 9.99;
bool isActive = true;
List<String> tags = ['dart', 'flutter'];
Map<String, dynamic> config = {'key': 'value', 'num': 42};

// Null safety
String? nullableName;           // Can be null
String nonNullName = 'value';    // Cannot be null
late String lateInitialized;     // Non-null, initialized later

// Null-aware operators
String display = nullableName ?? 'Default';
String upper = nullableName?.toUpperCase() ?? 'UNKNOWN';
int length = nullableName?.length ?? 0;

// Required and named parameters
void configure({
  required String host,
  int port = 8080,
  bool ssl = true,
}) {
  // Implementation
}

// Usage
configure(host: 'api.example.com');
configure(host: 'localhost', port: 3000, ssl: false);
```

### **Collections**

```dart
// List operations
final items = <String>['a', 'b', 'c'];
items.add('d');
items.insert(0, 'start');
final mapped = items.map((e) => e.toUpperCase()).toList();
final filtered = items.where((e) => e.length > 1).toList();
final first = items.firstWhere((e) => e.startsWith('a'), orElse: () => '');

// Spread and collection if/for
final extended = ['header', ...items, 'footer'];
final conditional = [
  'always',
  if (isActive) 'active',
  for (var i = 0; i < 3; i++) 'item-$i',
];

// Map operations
final user = {'name': 'John', 'age': 30};
user['email'] = 'john@example.com';
final name = user['name']; // Returns nullable
final hasKey = user.containsKey('name');

// Set (unique values)
final uniqueTags = <String>{'dart', 'flutter', 'dart'}; // {'dart', 'flutter'}
```

### **Functions**

```dart
// Arrow syntax (single expression)
int sum(int a, int b) => a + b;

// Anonymous functions
final list = [1, 2, 3];
list.forEach((item) => print(item));

// Closures
Function makeMultiplier(int multiplier) {
  return (int value) => value * multiplier;
}

var triple = makeMultiplier(3);
print(triple(5)); // 15

// Async/await
Future<String> fetchData() async {
  await Future.delayed(Duration(seconds: 1));
  return 'Data';
}

// Generator functions
Iterable<int> naturalsTo(int n) sync* {
  int k = 0;
  while (k < n) yield k++;
}

// Extension methods
extension StringExtension on String {
  String get reversed => split('').reversed.join('');
  bool get isBlank => trim().isEmpty;
}
```

---

## **49.2 Null Safety Migration Guide**

### **Understanding Null Safety**

```dart
// Pre-null safety (Dart 2.11 and earlier)
String name; // Could be null, could be string

// Post-null safety (Dart 2.12+)
String name; // Error: Non-nullable variable must be initialized
String? name; // OK: Explicitly nullable
late String name; // OK: Initialized later, non-nullable
```

### **Migration Strategy**

**Step 1: Prepare Dependencies**
```yaml
# pubspec.yaml
environment:
  sdk: '>=2.12.0 <4.0.0'  # Minimum Dart 2.12

dependencies:
  # Ensure all dependencies support null safety
  http: ^1.0.0  # Null-safe version
```

**Step 2: Automated Migration**
```bash
# Preview migration
dart migrate

# Apply migration
dart migrate --apply-changes
```

**Step 3: Manual Fixes**
```dart
// Before migration
class User {
  String name;
  int age;
  User({this.name, this.age});
}

// After migration
class User {
  String name;
  int? age;
  User({required this.name, this.age});
}

// Handling JSON with null safety
factory User.fromJson(Map<String, dynamic> json) {
  return User(
    name: json['name'] as String, // Cast required
    age: json['age'] as int?,     // Nullable cast
  );
}
```

### **Common Patterns**

```dart
// Defensive coding
void processUser(User? user) {
  // Early return
  if (user == null) return;
  
  // Null-aware cascade
  user
    ?..updateLastSeen()
    ..incrementLoginCount();
}

// Required fields in constructors
class Product {
  final String id;
  final String name;
  final String? description; // Optional
  
  Product({
    required this.id,
    required this.name,
    this.description,
  });
}

// Late initialization with validation
class Database {
  late final Connection _connection;
  
  Future<void> open() async {
    _connection = await Connection.create();
  }
  
  void query(String sql) {
    // Throws if accessed before open()
    _connection.execute(sql);
  }
}
```

---

## **49.3 Effective Dart Style Guide**

### **Naming Conventions**

```dart
// DO: Use camelCase for variables and functions
String firstName;
void processData() {}

// DO: Use PascalCase for classes and enums
class HttpRequest {}
enum Color { red, green, blue }

// DO: Use SCREAMING_SNAKE_CASE for constants
const double PI = 3.14159;
const String API_BASE_URL = 'https://api.example.com';

// DO: Use underscore prefix for private members
int _internalCounter;
void _privateMethod() {}

// DON'T: Use Hungarian notation
String strName; // Bad
int nCount;     // Bad
```

### **Documentation**

```dart
/// Calculates the distance between two points.
/// 
/// Uses the Pythagorean theorem to compute the straight-line
/// distance between [start] and [end].
/// 
/// Returns the distance in meters, rounded to 2 decimal places.
/// 
/// Example:
/// ```dart
/// final distance = calculateDistance(
///   Point(0, 0),
///   Point(3, 4),
/// ); // Returns 5.0
/// ```
double calculateDistance(Point start, Point end) {
  // Implementation
}

// Inline documentation for complex logic
// TODO(username): Refactor this to use Strategy pattern
// FIXME: This breaks when input is negative
// HACK: Temporary workaround for API limitation
```

### **Code Structure**

```dart
// Order of class members:
// 1. Static constants
// 2. Static variables
// 3. Instance variables (final first)
// 4. Constructors
// 5. Factory constructors
// 6. Static methods
// 7. Instance methods
// 8. Private helpers

class ApiClient {
  // 1. Static constants
  static const Duration defaultTimeout = Duration(seconds: 30);
  
  // 2. Static variables
  static final Map<String, ApiClient> _cache = {};
  
  // 3. Instance variables
  final String baseUrl;
  final HttpClient _client;
  String? _authToken;
  
  // 4. Constructor
  ApiClient._(this.baseUrl) : _client = HttpClient();
  
  // 5. Factory constructor
  factory ApiClient(String baseUrl) {
    return _cache.putIfAbsent(baseUrl, () => ApiClient._(baseUrl));
  }
  
  // 6. Static methods
  static void clearCache() => _cache.clear();
  
  // 7. Public instance methods
  Future<Response> get(String path) async {
    return _execute('GET', path);
  }
  
  // 8. Private helpers
  Future<Response> _execute(String method, String path) async {
    // Implementation
  }
}
```

---

## **49.4 Async Patterns Reference**

### **Future Patterns**

```dart
// Sequential execution
Future<void> processSequential() async {
  final data = await fetchData();
  final processed = await processData(data);
  await saveData(processed);
}

// Parallel execution
Future<void> processParallel() async {
  final futures = [
    fetchUserData(),
    fetchSettings(),
    fetchNotifications(),
  ];
  
  final results = await Future.wait(futures);
  final user = results[0];
  final settings = results[1];
  final notifications = results[2];
}

// Error handling with specific catch
Future<void> robustFetch() async {
  try {
    final data = await fetchData();
  } on SocketException catch (e) {
    // Network error
    print('Network error: $e');
  } on TimeoutException {
    // Timeout specific handling
    print('Request timed out');
  } catch (e, stackTrace) {
    // Catch-all
    print('Error: $e\n$stackTrace');
  } finally {
    // Always executes
    print('Cleanup');
  }
}

// Timeout wrapper
Future<String> fetchWithTimeout() {
  return fetchData().timeout(
    Duration(seconds: 5),
    onTimeout: () => 'Default data',
  );
}
```

### **Stream Patterns**

```dart
// Basic stream handling
Stream<int> numberStream() async* {
  for (var i = 0; i < 10; i++) {
    await Future.delayed(Duration(milliseconds: 100));
    yield i;
  }
}

// Stream transformations
void streamOperations() {
  final stream = numberStream();
  
  stream
    .where((n) => n.isEven)     // Filter
    .map((n) => n * n)          // Transform
    .take(3)                     // Limit
    .listen(
      (data) => print(data),
      onError: (e) => print('Error: $e'),
      onDone: () => print('Complete'),
    );
}

// Stream controllers
class EventBus {
  final _controller = StreamController.broadcast();
  
  Stream get stream => _controller.stream;
  
  void emit(dynamic event) {
    _controller.add(event);
  }
  
  void dispose() {
    _controller.close();
  }
}

// Async generators with yield*
Stream<int> combinedStream() async* {
  yield* numberStream();
  yield* Stream.fromIterable([100, 101, 102]);
}
```

### **Advanced Async**

```dart
// Completer for manual Future control
Future<String> manualFuture() {
  final completer = Completer<String>();
  
  Timer(Duration(seconds: 1), () {
    if (someCondition) {
      completer.complete('Success');
    } else {
      completer.completeError('Failed');
    }
  });
  
  return completer.future;
}

// Isolate communication
Future<String> heavyComputation() async {
  final receivePort = ReceivePort();
  
  await Isolate.spawn(_isolateFunction, receivePort.sendPort);
  
  final result = await receivePort.first;
  return result as String;
}

void _isolateFunction(SendPort sendPort) {
  // Heavy computation here
  final result = 'Computed result';
  sendPort.send(result);
}

// Zone error handling
runZonedGuarded(() async {
  await riskyOperation();
}, (error, stack) {
  print('Uncaught error: $error');
});
```

---

## **Chapter Summary**

This chapter provided a concise reference for Dart programming in Flutter:

### **Key Quick References:**

1. **Variables**: Use `var` with inference, `final` for immutability, `const` for compile-time constants. Prefer `??` and `?.` for null safety.

2. **Collections**: Use spread operators (`...`) and collection `if/for`. Prefer `map`/`where`/`fold` over manual loops.

3. **Functions**: Arrow syntax for single expressions. Named parameters with `required` for mandatory fields.

4. **Null Safety**: Use `?` for nullable, `late` for deferred initialization, `required` for constructor parameters. Migrate dependencies before code.

5. **Style**: `camelCase` for variables, `PascalCase` for classes, `SCREAMING_SNAKE_CASE` for constants. Order: constants → fields → constructors → methods.

6. **Async**: `await` for sequential, `Future.wait` for parallel. Always handle errors in async code. Use `async*`/`yield` for streams.

---

## **Next Chapter**

In **Chapter 50: Flutter Widget Catalog**, we will provide a comprehensive reference of Flutter widgets organized by category, including layout widgets, Material Design components, Cupertino widgets, and animation widgets with usage examples.

---

**End of Chapter 49**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='48. performance_optimization.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='50. package_ecosystem_guide.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
